# Using a Keras Image Generator in Amazon SageMaker

Amazon SageMaker is a fully-managed service that provides developers and data scientists with the ability to build, train, and deploy machine learning (ML) models quickly. Amazon SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high-quality models. The SageMaker Python SDK makes it easy to train and deploy models in Amazon SageMaker with several different machine learning and deep learning frameworks, including TensorFlow and Keras.

In this notebook, we train and host a MobileNet model using Keras. The model helps us to do a binary classification on an image.

In order reduce the code required, we will make use keras.preprocessing.image.ImageDataGenerator class. This class allows you to instantiate generators of image batches via a directory. These generators can then be used with the Keras model methods that accept data generators as inputs.

## Setup

First, we prepare the images by putting them into a training folder and 20% of images into a validation folder and upload them to an S3 bucket.

### Prepare the images

Download the images to the dataset folder. The folder structure is as such:
- dataset
    - training
        - classA
           - image1.jpg
           - image2.jpg
        - classB
           - image3.jpg
           - image4.jpg
    - validation
        - classA
           - image5.jpg
           - image6.jpg
        - classB
           - image7.jpg
           - image8.jpg

## Train the model

We train a CNN model with the dataset.


In [ ]:
import os
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlow
from sagemaker import get_execution_role
from tensorflow.python.keras.preprocessing.image import load_img


sagemaker_session = sagemaker.Session()
role = get_execution_role()
region = sagemaker_session.boto_session.region_name


bucket = "YOUR_BUCKET"
key = "dataset/images"

instance_type='ml.m5.xlarge' # The type of EC2 instance which will be used for training
local_hyperparameters={
    'epochs': 10,
    'batch-size' : 64
}

train_input_path = "s3://{}/{}/train/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

estimator = TensorFlow(entry_point='image_quality_keras_main.py',
                       source_dir='source_dir',
                       role=role,
                       framework_version='1.15.2',
                       py_version='py3',
                       hyperparameters=local_hyperparameters,
                       train_instance_count=1,
                       train_instance_type=instance_type)

estimator.fit({'training': train_input_path, 
               'validation': validation_input_path})




## Run predictions

Next we upload the model to a sagemaker endpoint and run predictions.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type,endpoint_type='tensorflow-serving')

In [ ]:
import numpy as np
import json
from matplotlib.pyplot import imshow, figure
from tensorflow.python.keras.preprocessing.image import load_img
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import load_img, img_to_array


    
test_image = load_img("images/image1.jpg", target_size=(224, 224))
test_image_array = np.array(test_image).reshape((1, 224, 224, 3))
figure()    
imshow(test_image)

scores = predictor.predict(test_image_array)
print(scores)




In [ ]:
predictor.delete_endpoint()